In [2]:
import numpy as np
import pymisca.util as pyutil;reload(pyutil)
import pymisca.vis_util as pyvis;reload(pyvis)
%matplotlib inline 

is in ipython: 1
is in ipython: 1
is in ipython: 1
[WARN] pymisca.vis_util cannot find network
[WARN] pymisca.vis_util cannot find network


In [1]:
#### Adapted from: http://kyleclo.github.io/maximum-likelihood-in-tensorflow-pt-1/

import numpy as np

D = 2


TRUE_MU = 0.0
TRUE_SIGMA = 2.0
SAMPLE_SIZE = (5000,D)

np.random.seed(0)
x_obs = np.random.normal(loc=TRUE_MU, scale=TRUE_SIGMA, size=SAMPLE_SIZE)
rsq = np.square(x_obs).sum(axis=-1)
keep = rsq > np.mean(rsq)
x_obs = x_obs[keep]

x_obs.T[0] = x_obs.T[0]*0.1

# x_obs = np.square(x_obs)
X = x_obs.astype(np.float32)

In [ ]:
tloc = 0.
def makeEMD( (mu,phi,sigma) ):
    dist_rsq   = tfdist.Gamma(mu,phi)
#     dist_rsq = tfbij.Invert(tfbij.Square()) (dist_r)

    dist_Theta = tfdist.Normal(tloc, 1.,)
    
    dist_Theta = pytf.VonMisesFisherCosine(D=2,concentration = 3.)

    dist_rsqTheta = mdl=  pytf.JointScalar(
        [
            tfdist.Gamma(mu,phi,name='r_squared'),
            dist_Theta,
        ])

    #### Use a bijector to calculate P(x) from P(r^2)
    dist_xyz = mdl = pytf.AsRadialTheta(distribution=mdl,D=D)
    
    ### Allow an affine transformation y = M x + x_0
#     dist_aff = mdl = pytf.AffineTransformDiag(distribution=dist_xyz,
#                                         scale_diag=sigma,)
    # gaussian_dist = tf.contrib.distributions.Normal(loc=mu, scale=sigma)
    # emission  = dist_rsq
    return mdl
# print em.distribution.event_shape
em = makeEMD((2., 1.,2.))
sess = tf.InteractiveSession()
f = em.prob
f = pyutil.wrapTFmethod(f)
f = pyutil.arrayFunc2mgridFunc(f)

with sess.as_default():
    pyvis.dmet_2d(f,vectorised=1);
    iY = em.bijector.inverse(X).eval()
    pyvis.qc_2var(iY.T[0],iY.T[1])
    pyutil.printlines(
        ('\int P(\theta) P(r^2) \partial x \partial y',
           pyutil.cubicRInt(f.arrayFunc,D=D,N=2000000,R=16.))
    )

    import scipy.stats as spstats
    f = spstats.norm._pdf
    pyutil.printlines(('\int P(\theta) \partial \theta',
    pyutil.surfInt(lambda x:f(x-tloc))))

